In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 76.7 MB/s 


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json
import numpy as np
from konlpy.tag import Okt 
from collections import Counter
import re

# Curating 업로드 날짜 / Curating 글 고유 번호 크롤링 

In [3]:
date = {
    '날짜' : []
}

In [4]:
url = 'https://www.musinsa.com/app/curating/lists'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
    'referer': 'https://www.musinsa.com/app/showcase/lists'
}
resp = requests.get(url,headers = headers)

soup = BeautifulSoup(resp.text,'lxml')
item_tags = soup.select('div.list_img_curating a')

item_num_tags = item_tags[0]['href'].replace('/app/curating/views/','')


In [5]:
total_page = 12951//60
total_page

215

In [6]:
num_dic={
    '큐레이팅번호':[]
}

In [7]:

for num in tqdm(range(1,total_page+2)):

  # url = 'https://www.musinsa.com/app/curating/lists?brand=&sort=new&event_yn=&display_cnt=60&page=1&tag=&editor=&total=12951'
  url = 'https://www.musinsa.com/app/curating/lists?'
  params = {
      'sort':'new',
      'display_cnt':60,
      'page': num,
      'total':12951
  }
  headers = {
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
  }

  resp = requests.get(url,params = params,headers=headers)
  soup = BeautifulSoup(resp.text,'lxml')
  item_tags = soup.select('div.list_img_curating a')
  for i in item_tags:
    num_dic['큐레이팅번호'].append(i['href'].replace('/app/curating/views/',''))


# [0]['href'].replace('/app/curating/views/','')

  0%|          | 0/216 [00:00<?, ?it/s]

In [8]:
len(num_dic['큐레이팅번호'])

12306

In [9]:
df = pd.DataFrame(num_dic['큐레이팅번호'])
df.columns=['Curating_Num']
df

,Curating_Num
0,23734
1,23732
2,23731
3,23730
4,23729
...,...
12301,752
12302,740
12303,730
12304,721


# Curating 번호, 날짜 DataFrame 만들기 

In [10]:
for i in tqdm(num_dic['큐레이팅번호']):

  url = 'https://www.musinsa.com/app/curating/views/'+ str(i)
  headers = {
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
      'referer': 'https://www.musinsa.com/app/curating/lists'
  }
  resp = requests.get(url,headers = headers)

  soup = BeautifulSoup(resp.text,'lxml')
  date_tag = soup.select('li.date span')
  date_tags = date_tag[0].text
  date['날짜'].append(date_tags.split(' ')[0])

date

  0%|          | 0/12306 [00:00<?, ?it/s]

{'날짜': ['2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.25',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.24',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.23',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.22',
  '2022.02.21',
  '2022.02.21',
  '2022.02.21',
  '2022.02.21',
  '2022.02.21',
  '2022.02.21',
  '2022.02.21',
  

In [11]:
df1 = pd.DataFrame(date)
df1.columns=['Date']
df1

,Date
0,2022.02.25
1,2022.02.25
2,2022.02.25
3,2022.02.25
4,2022.02.25
...,...
12301,2015.10.30
12302,2015.10.27
12303,2015.10.23
12304,2015.10.21


In [12]:
curating_date = pd.concat([df,df1],axis=1)
curating_date

,Curating_Num,Date
0,23734,2022.02.25
1,23732,2022.02.25
2,23731,2022.02.25
3,23730,2022.02.25
4,23729,2022.02.25
...,...,...
12301,752,2015.10.30
12302,740,2015.10.27
12303,730,2015.10.23
12304,721,2015.10.21


In [13]:
curating_date.to_csv('큐레이팅번호,날짜')

In [14]:
pd.read_csv('/content/큐레이팅번호,날짜',index_col=0)

,Curating_Num,Date
0,23734,2022.02.25
1,23732,2022.02.25
2,23731,2022.02.25
3,23730,2022.02.25
4,23729,2022.02.25
...,...,...
12301,752,2015.10.30
12302,740,2015.10.27
12303,730,2015.10.23
12304,721,2015.10.21
